https://golang.org/doc/codewalk/markov/
https://en.wikipedia.org/wiki/Markov_chain
https://en.wikipedia.org/wiki/N-gram#n-gram_models

In [ ]:
import nltk
import random

# create a dict with the ngram model, it receives a list with the samples as string
ngram_dict = {}

def create_ngram_dict(corpus):
    n = 3
    ngrams = nltk.ngrams(corpus, n)
    
    for grams in ngrams:
        dict_key = grams[:-1][0] + " " + grams[:-1][1]
        if dict_key in ngram_dict:
            ngram_dict[dict_key].append(grams[-1])
        else:
            ngram_dict[dict_key] = []
            ngram_dict[dict_key].append(grams[-1])

In [ ]:
def generate(seed, samples = 15):
    output = seed  
    for i in range(samples):
        # When it reaches the last prefix, there is no suffix, so end
        try:
            new_sample = random.choice(ngram_dict[seed])
        except:
            return output
        output += " " + new_sample
        seed = seed.split(" ")[1] + " " + new_sample

    return output

In [ ]:
word_corpus = 'am I a gram or am I a markov chain ... maybe I am both'
ngram_dict = {}
create_ngram_dict(word_corpus.split(" "))
ngram_dict

In [ ]:
print generate("am I", 100)

In [ ]:
import pickle

midi_dataset = pickle.load(open("validation_DB12_final_cleaned.p", "rb"))
print("ready!!!")

In [ ]:
corpus = midi_dataset["x"]
corpus_str=map(str, corpus)

In [ ]:
ngram_dict = {}
create_ngram_dict(corpus_str)
print("ready!!!")

In [ ]:
print generate("60 62", 100)